# Obtendo Histórico de Músicas do Leonardo:

In [1]:
# Importando Bibliotecas
import requests
import datetime
import spotipy
import spotipy.util as util
import ast
from typing import List
from os import listdir

In [2]:
# Referência: https://towardsdatascience.com/get-your-spotify-streaming-history-with-python-d5a208bbcbd3
username = 'leonardodmabreu'
client_id ='97a6154236f84c0f8e835a1171e30881'
client_secret = 'd4d778c7d61f409190187b360ef11b6b'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)
token

'BQBXHi29TypFUuoslVfNJpg8O-FvyFdDZ2c8kMVy_igogE-4bZ4khBXZuaLPoRD-IUpsKs-1cm-oxXtykw8cO4h75yQs6PwFCrcbZhzyGzV5MFX0lbhD3RdfM2iNJqp-Jwx5BERClwb9xRMl_nXKa1n8KbgVwv-o'

In [3]:
def get_id(track_name: str, token: str) -> str:
    headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer ' + token,
    }
    params = [
    ('q', track_name),
    ('type', 'track'),
    ]
    try:
        response = requests.get('https://api.spotify.com/v1/search', 
                    headers = headers, params = params, timeout = 5)
        json = response.json()
        first_result = json['tracks']['items'][0]
        track_id = first_result['id']
        return track_id
    except:
        return None
    
# TESTE
lucy_id = get_id('Beautiful Day', token)
print(lucy_id)

0gzqZ9d1jIKo9psEIthwXe


In [4]:
def get_features(track_id: str, token: str) -> dict:
    sp = spotipy.Spotify(auth=token)
    try:
        features = sp.audio_features([track_id])
        return features[0]
    except:
        return None
    
# TESTE
lucy_features = get_features(lucy_id, token)
print(lucy_features)

{'danceability': 0.536, 'energy': 0.928, 'key': 11, 'loudness': -6.492, 'mode': 0, 'speechiness': 0.0589, 'acousticness': 0.0154, 'instrumentalness': 0.00134, 'liveness': 0.2, 'valence': 0.389, 'tempo': 136.258, 'type': 'audio_features', 'id': '0gzqZ9d1jIKo9psEIthwXe', 'uri': 'spotify:track:0gzqZ9d1jIKo9psEIthwXe', 'track_href': 'https://api.spotify.com/v1/tracks/0gzqZ9d1jIKo9psEIthwXe', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0gzqZ9d1jIKo9psEIthwXe', 'duration_ms': 248400, 'time_signature': 4}


In [5]:
# Função que coleta os dados do histórico do Spotify, extrai o JSON e converte em dicionários
def get_streamings(path: str = 'MyData') -> List[dict]:
    
    files = ['MyData/' + x for x in listdir(path)
             if x.split('.')[0][:-1] == 'NaoCurtidas']
    
    all_streamings = []
    
    for file in files: 
        with open(file, 'r', encoding='UTF-8') as f:
            new_streamings = ast.literal_eval(f.read())
            all_streamings += [streaming for streaming 
                               in new_streamings]
    return all_streamings


streamings = get_streamings()
unique_tracks = list(set([streaming['track'] 
                for streaming in streamings]))

all_features = {}
for track in unique_tracks:
    track_id = get_id(track, token)
    features = get_features(track_id, token)
    if features:
        all_features[track] = features
        
with_features = []
for track_name, features in all_features.items():
    with_features.append({'name': track_name, **features})

# TRANSFORMANDO EM CSV:

In [6]:
import pandas as pd
df = pd.DataFrame(with_features)
df.to_csv('nao_curtidas.csv')